# Intuition on AURC

This notebooks contains examples of how to use the failure prediction modules.
We measure metrics of interest on:
- CIFAR-10 (ResNet-18)

We use the following metrics:
- Accuracy
- F1 score (micro,macro)
- ECE
- Brier score
- Negative Loglikelihood
- AURC
- plot of risk-coverage curves

1) We first re-implement the AURC metric from "BIAS-REDUCED UNCERTAINTY ESTIMATION FOR DEEP NEURAL CLASSIFIERS"(https://arxiv.org/pdf/1805.08206.pdf). 
   
   We use the version/implementation from "A CALL TO REFLECT ON EVALUATION PRACTICES FOR FAILURE DETECTION IN IMAGE CLASSIFICATION"(https://openreview.net/pdf?id=YnkGMIh0gvX)
   
   > Geifman et al. steps in the RC curve are not defined as new unique values in the sorted list of
confidence scores, but instead each data sample, including the ones with equal confidence scores, is
considered as leading to an individual classification decision with an associated risk-coverage pair.
This effectively leads to a random interpolation of the RC-curve between unique confidence scores
adding considerable noise to the result especially in dense singular points such as confidence scores
of 0 or 1. As a result, the AURC metric is not well defined and can lead to different results depending on the implementation. 


1) Then, we implement a 'naive' version based on Definition 2. 
    We will test for different N if 

Next, we implement a vectorized version. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## necessary installs
!pip3 install numpy
!pip3 install scipy
!pip3 install scikit-learn
!pip3 install matplotlib
!pip3 install pandas
!pip3 install plotly

^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/home/jordy/.virtualenvs/SOTA/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/jordy/.virtualenvs/SOTA/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/jordy/.virtualenvs/SOTA/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/jordy/.virtualenvs/SOTA/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/jordy/.virtualenvs/SOTA/bin/python -m pip install --upgrade pip' command.


## Measuring calibration on CIFAR-10


In [9]:
"""
This testing script loads actual probabilisitic predictions from a resnet finetuned on CIFAR

There are a number of logits-groundtruth pickles available @ https://github.com/markus93/NN_calibration/tree/master/logits
[Seems to have moved from Git-LFS to sharepoint]
https://tartuulikool-my.sharepoint.com/:f:/g/personal/markus93_ut_ee/EmW0xbhcic5Ou0lRbTrySOUBF2ccSsN7lo6lvSfuG1djew?e=l0TErb

See https://github.com/markus93/NN_calibration/blob/master/logits/Readme.txt to decode the [model_dataset] filenames

As a bonus, one could consider temperature scaling and measuring after calibration.
"""
from argparse import Namespace
import numpy as np
import math
from scipy.special import softmax
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc


# Open file with pickled variables
def unpickle_probs(file, verbose=0, normalize=False):
    with open(file, "rb") as f:  # Python 3: open(..., 'rb')
        y1, y2 = pickle.load(f)  # unpickle the content

    if isinstance(y1, tuple):
        y_probs_val, y_val = y1
        y_probs_test, y_test = y2
    else:
        y_probs_val, y_probs_test, y_val, y_test = train_test_split(
            y1, y2.reshape(-1, 1), test_size=len(y2) - 5000, random_state=15
        )  # Splits the data in the ca%load_ext autoreload

    if normalize:
        y_probs_val = softmax(y_probs_val, -1)
        y_probs_test = softmax(y_probs_test, -1)

    if verbose:
        print("y_probs_val:", y_probs_val.shape)  # (5000, 10); Validation set probabilities of predictions
        print("y_true_val:", y_val.shape)  # (5000, 1); Validation set true labels
        print("y_probs_test:", y_probs_test.shape)  # (10000, 10); Test set probabilities
        print("y_true_test:", y_test.shape)  # (10000, 1); Test set true labels

    return ((y_probs_val, y_val.ravel()), (y_probs_test, y_test.ravel()))

In [10]:
import pandas as pd
import json
from collections import OrderedDict
from metrics import accuracy, brier_loss, nll, f1_micro, f1_macro, aurc_logits, ece_logits  # AUROC_logits

METRICS = [accuracy, brier_loss, nll, f1_micro, f1_macro, ece_logits, aurc_logits]


def apply_metrics(y_true, y_probs, metrics=METRICS):
    predictive_performance = OrderedDict()
    for metric in metrics:
        try:
            predictive_performance[f"{metric.__name__.replace('_logits', '')}"] = metric(y_true, y_probs)
        except Exception as e:
            print(e)
    print(json.dumps(predictive_performance, indent=4))
    return predictive_performance


(p_val, y_val), (p_test, y_test) = unpickle_probs("../data/resnet110_c10_logits.p", verbose=1)
output = apply_metrics(y_test, p_test)
#
df = pd.DataFrame.from_dict(output, orient="index", columns=["resnet110_c10"])
print(df.to_latex())

def test_aurc():
    """
    Three cases from https://openreview.net/pdf?id=YnkGMIh0gvX

    separable_less_accurate_references ; acc 2/5, AUROC 1
    unseparable_lowcorrect_references ; acc 3/5, AUROC 0.75
    unseparable_highincorrect_references ; acc 3/5, AUROC 0.583
    """
    g_X = np.array([0.9, 0.1, 0.3, 1.0, 0.1])
    separable_less_accurate_references = np.array([1, 0, 0, 1, 0])
    unseparable_lowcorrect_references = np.array([1, 1, 0, 1, 0])
    unseparable_highincorrect_references = np.array([0, 1, 1, 1, 0])


def simple_testcase(add_nonunique=False):
    p_test = np.array([[0.6, 0.2, 0.2], [0, 0.95, 0.05], [0.7, 0.1, 0.2], [0.99, 0.01, 0]]) * 5  # Z to make it logits
    y_test = np.array([0, 1, 2, 0])
    if add_nonunique:
        p_test = np.vstack([p_test, p_test])
        y_test = np.hstack([y_test, y_test])
    return p_test, y_test

p_simp, y_simp = simple_testcase()
# p_simp, y_simp = simple_testcase(add_nonunique=True)

y_probs_val: (5000, 10)
y_true_val: (5000, 1)
y_probs_test: (10000, 10)
y_true_test: (10000, 1)
{
    "accuracy": 0.9356,
    "brier_loss": 0.11018865559514414,
    "nll": 1.0624524426782735,
    "f1_micro": 0.9356,
    "f1_macro": 0.9356251148224297,
    "ece": 0.05030814408063887,
    "aurc": 0.008406399931071139
}
\begin{tabular}{lr}
\toprule
{} &  resnet110\_c10 \\
\midrule
accuracy   &       0.935600 \\
brier\_loss &       0.110189 \\
nll        &       1.062452 \\
f1\_micro   &       0.935600 \\
f1\_macro   &       0.935625 \\
ece        &       0.050308 \\
aurc       &       0.008406 \\
\bottomrule
\end{tabular}



## AURC (official implementation)

In [19]:
# g = lambda x: np.max(softmax(x, axis=-1), -1)  # maximum softmax probability


def entropy(x):
    exp_x = np.exp(x)
    A = np.sum(exp_x, axis=-1)  # sum of exp(x_i)
    B = np.sum(x * exp_x, axis=-1)  # sum of x_i * exp(x_i)
    return np.log(A) - B / A


g = lambda x: -entropy(x)  # negative entropy to not have to deal with sign


def coverage_at_threshold(f_X, g, tau=0.5):
    return np.mean(g(f_X) >= tau)


def risk_at_threshold(f_X, g, Y, tau=0.5, loss="0-1"):
    return np.logical_and((g(f_X) >= tau), (f_X.argmax(-1) != Y)).mean()


# for t in [0.2, 0.5, 0.8, 0.9, 0.95, 0.97, 0.99]:
#     print(f"Risk at {t}: {risk_at_threshold(p_test, g, y_test, tau=t)}")
#     print(f"Coverage at {t}: {coverage_at_threshold(p_test, g, tau=t)}")

# now defining AURC as a curve


def geifman_AURC(f_X, g, Y, plot=False):
    residuals = f_X.argmax(-1) != Y
    confidence = g(f_X)
    curve = []
    m = len(residuals)
    idx_sorted = np.argsort(confidence)
    temp1 = residuals[idx_sorted]
    cov = len(temp1)
    acc = sum(temp1)
    curve.append((cov / m, acc / len(temp1)))
    for i in range(0, len(idx_sorted) - 1):
        cov = cov - 1
        acc = acc - residuals[idx_sorted[i]]
        curve.append((cov / m, acc / (m - i)))
    AUC = sum([a[1] for a in curve]) / len(curve)
    err = np.mean(residuals)
    kappa_star_aurc = err + (1 - err) * (np.log(1 - err))
    EAURC = AUC - kappa_star_aurc
    coverages, risks = zip(*curve)
    weights = [1 / m] * (
        len(coverages) - 1
    )  # just forms some mask of points that were different/used for integration (x/N) --> with percentage of data captured
    # make it like the version of Jaeger
    # weights = [coverages[i] - coverages[i+1] for i in range(len(coverages)-1)] --> same result as above
    weights[0] = 0
    test_weights = True
    if test_weights:
        AUC2 = sum(
            [(risks[i] + risks[i + 1]) * 0.5 * weights[i] for i in range(len(weights))]
        )  # area of trapezoid (weights are constant...)
        print(AUC, AUC2, math.isclose(AUC, AUC2))

    cache = Namespace(risks=risks, coverages=coverages, weights=weights)
    if plot:
        pd.options.plotting.backend = "plotly"
        df = pd.DataFrame(zip(coverages, risks, weights), columns=["% Coverage", "% Risk", "weights"])
        fig = df.plot(x="% Coverage", y="% Risk")
        fig.show()
    return {"aurc": AUC, "e-aurc": EAURC, "cache": cache}


def official_AURC(f_X, g, Y, plot=False):
    "The RC curve is obtained by computing the risk of the coverage from the beginning of g(x) (most confident) to the end (least confident)."
    incorrect = f_X.argmax(-1) != Y  # instance-level mask
    g_X = g(f_X)
    idx_sorted = np.argsort(g_X)  # in ascending format; construct curve from right to left

    coverages, risks = [], []
    weights = (
        []
    )  # just forms some mask of points that were different/used for integration (x/N) --> with percentage of data captured

    # well-defined starting point: risk = 1-accuracy (loss), coverage=100%; threshold=0
    coverages.append(1)
    risks.append(incorrect.mean())

    # will keep these as intermediate absolute values to facilitate calculation
    N = len(idx_sorted)
    coverage = len(idx_sorted)
    error_sum = sum(incorrect[idx_sorted])

    tmp_weight = 0
    for tau in range(0, len(idx_sorted) - 1):  # each
        coverage = coverage - 1
        error_sum = error_sum - incorrect[idx_sorted[tau]]
        selective_risk = error_sum / (N - 1 - tau)
        tmp_weight += 1
        if tau == 0 or g_X[idx_sorted[tau]] != g_X[idx_sorted[tau - 1]]:  # unique or starting threshold
            coverages.append(coverage / N)
            risks.append(selective_risk)
            weights.append(tmp_weight / N)
            tmp_weight = 0

    # well-defined ending (if not already done): last known risk for 0 coverage (threshold=100%)
    if tmp_weight > 0:
        coverages.append(0)
        risks.append(risks[-1])
        weights.append(tmp_weight / N)  # rest of the data

    # print(f"risk at coverages:{list(zip(risks, coverages))}")
    cache = Namespace(risks=risks, coverages=coverages, weights=weights)

    # print(len(risks), len(coverages), len(weights))
    test_weights = False
    if test_weights:
        checks = [math.isclose(weights[i], (coverages[i] - coverages[i + 1])) for i in range(len(weights))]
        print(all(checks))
        for i in range(len(weights)):
            if not checks[i]:
                print(i, weights[i], coverages[i] - coverages[i + 1], risks[i + 1] - risks[i])

    aurc = sum([(risks[i] + risks[i + 1]) * 0.5 * weights[i] for i in range(len(weights))])
    # aurc = auc(coverages, risks) #makes no difference
    if plot:
        pd.options.plotting.backend = "plotly"
        df = pd.DataFrame(zip(coverages, risks, weights), columns=["% Coverage", "% Risk", "weights"])
        fig = df.plot(x="% Risk", y="% Coverage")
        fig.show()
    return {"aurc": aurc, "cache": cache}


official = official_AURC(p_test, g, y_test, plot=True)
print(f"Geifman AURC: {geifman_AURC(p_test, g, y_test,plot=False)}")
print(f"Official AURC: {official}")

print(list(zip(official["cache"].coverages, official["cache"].risks, official["cache"].weights)))

0.008268754710453118 0.008265534710453097 False
Geifman AURC: {'aurc': 0.008268754710453118, 'e-aurc': 0.006149068451442143, 'cache': Namespace(coverages=(1.0, 0.9999, 0.9998, 0.9997, 0.9996, 0.9995, 0.9994, 0.9993, 0.9992, 0.9991, 0.999, 0.9989, 0.9988, 0.9987, 0.9986, 0.9985, 0.9984, 0.9983, 0.9982, 0.9981, 0.998, 0.9979, 0.9978, 0.9977, 0.9976, 0.9975, 0.9974, 0.9973, 0.9972, 0.9971, 0.997, 0.9969, 0.9968, 0.9967, 0.9966, 0.9965, 0.9964, 0.9963, 0.9962, 0.9961, 0.996, 0.9959, 0.9958, 0.9957, 0.9956, 0.9955, 0.9954, 0.9953, 0.9952, 0.9951, 0.995, 0.9949, 0.9948, 0.9947, 0.9946, 0.9945, 0.9944, 0.9943, 0.9942, 0.9941, 0.994, 0.9939, 0.9938, 0.9937, 0.9936, 0.9935, 0.9934, 0.9933, 0.9932, 0.9931, 0.993, 0.9929, 0.9928, 0.9927, 0.9926, 0.9925, 0.9924, 0.9923, 0.9922, 0.9921, 0.992, 0.9919, 0.9918, 0.9917, 0.9916, 0.9915, 0.9914, 0.9913, 0.9912, 0.9911, 0.991, 0.9909, 0.9908, 0.9907, 0.9906, 0.9905, 0.9904, 0.9903, 0.9902, 0.9901, 0.99, 0.9899, 0.9898, 0.9897, 0.9896, 0.9895, 0.9894, 0.9

In [56]:
## version with pairwise comparisons


def AURC_naive(f_X, g, Y, plot=False):
    """math
    \approx \frac{1}{n} \sum_{h=1}^n
    \frac{\frac{1}{n-1} \sum_{i\neq h} \ell([f(x_i)]_{1:k},y_i) \mathbb{I}[f(x_i)_{k+1} > f(x_h)_{k+1}]}
    {\frac{1}{n-1} \sum_{i\neq h} \mathbb{I}[f(x_i)_{k+1} > f(x_h)_{k+1}]}
    """
    risks = np.zeros(len(f_X))
    coverages = np.zeros(len(f_X))

    N = len(Y)
    outer_sum = 0
    for h in range(N):
        num = 0  # risk
        den = 0  # coverage
        for i in range(N):  # inner sums
            if i == h:
                continue
            loss = int(f_X[i].argmax(-1) != Y[i])
            indicator = g(f_X[i]) > g(f_X[h])
            num += loss * indicator  # empirical risk for h vs. i
            den += indicator  # empirical coverage for h vs. i

        risks[h] = num / (N - 1)
        coverages[h] = den / (N - 1)

        if den == 0:  # no coverage, no risk?
            # print('Comparison for h={} has 0 denominator'.format(h)) #which means it is max?
            continue
        outer_sum += (num / den)  # empirical risk for h vs. all i
    return outer_sum / N  # , risks, coverages


official = official_AURC(p_simp, g, y_simp, plot=False)
naive = AURC_naive(p_simp, g, y_simp)  # Definition 2
print(f"Geifman AURC: {geifman_AURC(p_simp, g, y_simp)}")
print(f"official: {official}")
print(f"naive: {naive}")

0.125 0.09375 False
Geifman AURC: {'aurc': 0.125, 'e-aurc': 0.09076155433883568, 'cache': Namespace(coverages=(1.0, 0.75, 0.5, 0.25), risks=(0.25, 0.25, 0.0, 0.0), weights=[0.25, 0.25, 0.25])}
official: {'aurc': 0.11458333333333331, 'cache': Namespace(coverages=[1, 0.75, 0.5, 0.25], risks=[0.25, 0.3333333333333333, 0.0, 0.0], weights=[0.25, 0.25, 0.25])}
naive: 0.08333333333333333


In [57]:
# AURC alphas

# now let's make a vectorized/matrix version that is more efficient to compute
def AURC_naive_alphas(f_X, g, Y, plot=False):
    '''AURC_naive alphas
    \frac{1}{n} \sum_{i=1}^{n-1} \frac{\ell([f(x_{i+1})]_{1:k},y_{i+1})}{(n-i)}
    '''
    g_X = g(f_X)
    indices_sorted = np.argsort(g_X)
    N = len(Y)
    losses = f_X.argmax(-1) != Y
    alphas = []
    for i in range(N):
        
    
    alphas = [1 / (N - indices_sorted[i] + 1) for i in range(N)]
    return np.mean(alphas * losses)


naive_alphas = AURC_naive_alphas(p_simp, g, y_simp)
naive_alphas

0.0

In [55]:
import plotly.graph_objects as go

np.random.seed(42)
import time


def runtime_wrapper(input_function, *args, **kwargs):
    start_value = time.perf_counter()
    return_value = input_function(*args, **kwargs)
    end_value = time.perf_counter()
    runtime_value = end_value - start_value
    # print(f"Finished executing {input_function.__name__} in {runtime_value} seconds")
    return return_value, runtime_value


implementations = [
    ("geifman", lambda *x: geifman_AURC(*x)["aurc"]),
    ("jaeger 2023", lambda *x: official_AURC(*x)["aurc"]),
    ("naive", AURC_naive),
    ("naive_alphas", AURC_naive_alphas),
]

consistency_results = []
timings = []
ns = [5, 10, 50, 100, 250, 500]  # , 1000, 2500, 5000, 10000]
for n in ns:
    sub_n = np.random.choice(list(range(len(y_test))), n)
    p_test_sub, y_test_sub = p_test[sub_n], y_test[sub_n]
    res_n = []
    timings_n = []
    for _, impl in implementations:
        v, t = runtime_wrapper(impl, p_test_sub, g, y_test_sub)
        res_n.append(v)
        timings_n.append(t)
    consistency_results.append(res_n)
    timings.append(timings_n)

impl_results = list(zip(*consistency_results))

fig = go.Figure()
for i, (key, _) in enumerate(implementations):
    fig.add_trace(go.Scatter(x=ns, y=impl_results[i], name=key))
fig.show()

timings_results = list(zip(*timings))
for i, (key, _) in enumerate(implementations):
    timings_per_n = list(zip(ns, timings_results[i]))
    # calculate gradient for timing with respect to n
    timings_per_n = [(n, t, t / n) for n, t in timings_per_n]
    print(f"{key}: {timings_per_n}")

0.08 0.06000000000000001 False
0.03111111111111111 0.026111111111111113 False
0.017032850901831667 0.01563285090183167 False
0.011755012160192833 0.011355012160192843 False
0.010579833224417669 0.01041983322441767 False
0.00637002554916861 0.0063160255491686084 False


geifman: [(5, 0.0004796129996975651, 9.592259993951302e-05), (10, 0.0004233409999869764, 4.233409999869764e-05), (50, 0.0011834090000775177, 2.3668180001550355e-05), (100, 0.0006268370016186964, 6.2683700161869635e-06), (250, 0.001455474999602302, 5.821899998409208e-06), (500, 0.0024354270008188905, 4.870854001637781e-06)]
jaeger 2023: [(5, 0.0001231620008184109, 2.4632400163682176e-05), (10, 0.00013728799967793748, 1.3728799967793747e-05), (50, 0.0004983569997421, 9.967139994842e-06), (100, 0.0008748690015636384, 8.748690015636385e-06), (250, 0.001510169000539463, 6.040676002157852e-06), (500, 0.002664658000867348, 5.329316001734696e-06)]
naive: [(5, 0.000637414999800967, 0.00012748299996019342), (10, 0.002878190000046743, 0.0002878190000046743), (50, 0.08253865799997584, 0.0016507731599995168), (100, 0.3235735279995424, 0.0032357352799954244), (250, 1.9030073819994868, 0.007612029527997948), (500, 7.390119794999919, 0.014780239589999838)]
naive_alphas: [(5, 6.127899905550294e-05, 1.2

In [35]:
## TODO: ongoing implementation of AURC using matrix operations
# https://pastebin.com/73xKmyzn


# now let's make a vectorized/matrix version that is more efficient to compute
def AURC_vectorized(f_X, g, Y, plot=False):
    """math
    \approx \frac{1}{n} \sum_{h=1}^n
    \frac{\frac{1}{n-1} \sum_{i\neq h} \ell([f(x_i)]_{1:k},y_i) \mathbb{I}[f(x_i)_{k+1} > f(x_h)_{k+1}]}
    {\frac{1}{n-1} \sum_{i\neq h} \mathbb{I}[f(x_i)_{k+1} > f(x_h)_{k+1}]}
    """
    losses = f_X.argmax(-1) != Y
    g_X = np.asarray(g(f_X))
    mask = g_X[:, None] >= g_X
    out = mask[~np.eye(g_X.size, dtype=bool)]  # ignore diagonal; select other elements

    num = losses * out  # element-wise multiplication
    den = out.sum() / len(Y)  # element-wise division

    # need to compute the risk and coverage for each element in the matrix
    ## numerator = sum of losses * mask
    ## denominator = sum of mask
    # numerator = np.sum(losses[out; None])
    # denominator = np.sum(out) / len(Y)
    return (num / den).mean()

    # pairwise comparison using numpy broadcasting
    # https://stackoverflow.com/questions/39479578/python-pairwise-comparison-of-elements-in-a-array-or-list


# AURC_vectorized(p_test, g, y_test)
# still O(N^2) complexity, but much faster than the naive version

AURC_vectorized(p_simp, g, y_simp)  # N=4. K=3

ValueError: operands could not be broadcast together with shapes (4,) (12,) 

In [43]:
# let's create a version where the integration points (weights) are defined in advance
# TODO: make sure to evaluate the curve at the right points (i.e. not just the ones that are in the dataset); unique trapezoids


def equalmass_AURC(f_X, g, Y, NUM_AUC_VALUES=10, plot=False):
    incorrect = f_X.argmax(-1) != Y  # instance-level mask
    g_X = g(f_X)

    percentiles = np.linspace(0, 100, NUM_AUC_VALUES + 1)  # always add 1
    integration_points = np.unique(np.percentile(g_X, percentiles))  # divide CSF into unique bins/integration points

    # add 0 and 1 to integration points if not already there?

    # Initialize variables for calculation
    bin_coverages = np.zeros(NUM_AUC_VALUES)
    bin_risks = np.zeros(NUM_AUC_VALUES)
    bin_freqs = np.zeros(NUM_AUC_VALUES)

    # Assign predictions to bins and calculate coverage and risk
    for j in range(len(integration_points) - 1):  # one less
        threshold_mask = g_X >= integration_points[j]
        bin_mask = np.logical_and(
            threshold_mask, g_X < integration_points[j + 1]
        )  # all above threshold but below next threshold
        bin_coverages[j] = np.nanmean(threshold_mask)  # all above threshold
        bin_risks[j] = np.nanmean(incorrect[threshold_mask])  # %error above threshold
        bin_freqs[j] = np.nanmean(
            bin_mask
        )  # should be a constant value thanks to percentiled approximation (diff between continuous)

    if sum(bin_freqs) != 1.0:  # final bin if necessary
        # all that are equal or larger than last integration point should be added
        threshold_mask = g_X >= integration_points[-1]
        bin_coverages[-1] = np.nanmean(threshold_mask)  # all above threshold
        bin_risks[-1] = np.nanmean(incorrect[threshold_mask])  # %error above threshold
        bin_freqs[-1] = np.nanmean(threshold_mask)  # should be a constant value thanks to percentiled approximation

    print(f"Integration points: {integration_points}")
    print(f"risk at coverages:{list(zip(bin_risks, bin_coverages, bin_freqs))}")
    assert np.isclose(sum(bin_freqs), 1.0)

    # now define curve here
    aurc = sum(
        [(bin_risks[i] + bin_risks[min(i + 1, len(bin_freqs) - 1)]) * 0.5 * bin_freqs[i] for i in range(len(bin_freqs))]
    )
    if plot:
        pd.options.plotting.backend = "plotly"
        df = pd.DataFrame(zip(bin_coverages, bin_risks, bin_freqs), columns=["% Coverage", "% Risk", "weights"])
        fig = df.plot(x="% Coverage", y="% Risk")
        fig.show()
    cache = Namespace(risks=bin_risks, coverages=bin_coverages, weights=bin_freqs)
    return {"aurc": aurc, "cache": cache}


equalmass_AURC(p_test, g, y_test, plot=True)

Integration points: [0.2957992  0.98766466 0.99970332 0.99997556 0.99999583 0.99999893
 0.99999964 0.99999988 1.        ]
risk at coverages:[(0.0644, 1.0, 0.1), (0.025333333333333333, 0.9, 0.1), (0.012, 0.8, 0.0997), (0.006568613451377981, 0.7003, 0.0991), (0.004491017964071856, 0.6012, 0.098), (0.0021860095389507153, 0.5032, 0.0811), (0.001658374792703151, 0.4221, 0.0622), (0.0013892747985551543, 0.3599, 0.0721), (0.0, 0.0, 0.0), (0.0010423905489923557, 0.2878, 0.2878)]


{'aurc': 0.008754912841592782,
 'cache': Namespace(coverages=array([1.    , 0.9   , 0.8   , 0.7003, 0.6012, 0.5032, 0.4221, 0.3599,
        0.    , 0.2878]), risks=array([0.0644    , 0.02533333, 0.012     , 0.00656861, 0.00449102,
        0.00218601, 0.00165837, 0.00138927, 0.        , 0.00104239]), weights=array([0.1   , 0.1   , 0.0997, 0.0991, 0.098 , 0.0811, 0.0622, 0.0721,
        0.    , 0.2878]))}

In [44]:
equalmass_AURC(p_test, g, y_test, NUM_AUC_VALUES=len(np.unique(g(p_test))), plot=True)

Integration points: [0.2957992  0.3878758  0.42066013 ... 0.99999976 0.99999988 1.        ]
risk at coverages:[(0.0644, 1.0, 0.0004), (0.06402561024409764, 0.9996, 0.0003), (0.06384469128389873, 0.9993, 0.0004), (0.06356992691961157, 0.9989, 0.0003), (0.06338874424193872, 0.9986, 0.0004), (0.0632137848126628, 0.9982, 0.0003), (0.0631325784146708, 0.9979, 0.0004), (0.06285714285714286, 0.9975, 0.0003), (0.06277577216205375, 0.9972, 0.0003), (0.0625940415287391, 0.9969, 0.0004), (0.06231811339688911, 0.9965, 0.0003), (0.06203573579602489, 0.9962, 0.0004), (0.06175938943562964, 0.9958, 0.0003), (0.06157709693621296, 0.9955, 0.0004), (0.061501356647573106, 0.9951, 0.0003), (0.0612183353437877, 0.9948, 0.0003), (0.06113624937154349, 0.9945, 0.0004), (0.061060255507494214, 0.9941, 0.0003), (0.06097806399678004, 0.9938, 0.0004), (0.06080128850412724, 0.9934, 0.0003), (0.060718960829725104, 0.9931, 0.0004), (0.06074342701722575, 0.9927, 0.0003), (0.0605602579604998, 0.9924, 0.0003), (0.0603769

{'aurc': 0.008369711600304032,
 'cache': Namespace(coverages=array([1.    , 0.9996, 0.9993, ..., 0.    , 0.    , 0.2878]), risks=array([0.0644    , 0.06402561, 0.06384469, ..., 0.        , 0.        ,
        0.00104239]), weights=array([0.0004, 0.0003, 0.0004, ..., 0.    , 0.    , 0.2878]))}

In [ ]:
# experiment with many randomly sampled dirichlet distribution values; try with different N

# Running example

#### The original data comes from [Gupta & Ramdas 2021](https://openreview.net/pdf?id=WqoBaaPHS-)

In [9]:
import numpy as np

f = np.array(
    [
        [0.1, 0.0, 0.6, 0.3],
        [0.6, 0.0, 0.1, 0.3],
        [0.2, 0.7, 0.0, 0.1],
        [0.0, 0.1, 0.1, 0.8],
        [0.0, 0.1, 0.8, 0.1],
        [0.9, 0.1, 0.0, 0.0],
    ]
)

confidence = np.array([[0.6, 0.6, 0.7, 0.8, 0.8, 0.9]])

predicted_y = np.array([3, 1, 2, 4, 3, 1])
correct_y = np.array([3, 4, 2, 1, 4, 1])
y_correct = correct_y - 1  # 0-indexing

## Risk-coverage curves


In [19]:
aurcs = []
caches = []
for probs in [p_test, scaled_test_p]:
    res = aurc_logits(y_test, probs, plot=False, get_cache=True)  # resnet
    aurc, cache = res["aurc"], res["cache"]
    aurcs.append(aurc)
    caches.append(cache)

multi_aurc_plot(caches, ["CIFAR10-ResNet18", "CIFAR10-ResNet18+T"], aurcs=aurcs)

## Differentiable version

torch.sort 
torch.sigmoid on the difference of f(x)_i and f(x)_h

In [ ]:
# .backward on the function with certain Tensor inputs